In [ ]:
import jieba.posseg as pseg
import torch
from torch.utils import data
import numpy as np

wordslist=['a','ad','an','c','d','f','i','m','n','nr','ns','nt','o','p','q','r','tg','t','u','v','vd','vn','w','x','y','z']
dim = len(wordslist)

In [ ]:
def LrTest(net,loss,features,labels):
    return loss(net(features),labels)

In [ ]:
def TextToVector(sentences):
    length = len(sentences)
    X = torch.zeros((length,dim),dtype=torch.float)

    cnt = 0
    for sentence in sentences:
        words = pseg.cut(sentence)
        for word, pos in words:
            if pos in wordslist:
                X[cnt][wordslist.index(pos)] += 1

        cnt += 1

    return X


In [ ]:
def load_array(data_arrays, batch_size, is_train=False):
    # 构造⼀个PyTorch数据迭代器
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [ ]:
import pandas as pd
df = pd.read_excel('年龄较大孩子答案.xlsx')
text_data = df['案件描述']  # 使用列标签
score_data = df['可信度得分']

featuresA = TextToVector(text_data)
labelsA = torch.tensor(score_data,dtype=torch.float)


In [ ]:
batch_size = 5
data_iterA = load_array((featuresA, labelsA), batch_size)

In [ ]:
from torch import nn
netA = nn.Sequential(nn.Linear(dim, 1))
lossA = nn.MSELoss()
trainerA = torch.optim.SGD(netA.parameters(), lr=0.03)

In [ ]:
sum_epochs = 3
for epoch in range(sum_epochs):
    for X, y in data_iterA:
        l = lossA(netA(X) ,y)
        trainerA.zero_grad()
        l.backward()
        trainerA.step()
    l = lossA(netA(featuresA), labelsA)
    print(f'epoch {epoch + 1}, loss {l:f}')

In [ ]:
print(LrTest(netA,lossA,featuresA,labelsA))


下面开始构建B模型

In [ ]:
import jieba.analyse
TopkeyCnt = 10

In [ ]:
typecnt = 6
dfkey = pd.read_excel('关键词库.xlsx')
keywordlist = []
for i in range(typecnt):
    x = dfkey.iloc[:, i].tolist()
    keywordlist.append({key: None for key in x})

In [ ]:
def TextToKeyVector(sentences):
    length = len(sentences)
    X = torch.zeros((length,typecnt),dtype=torch.float)

    cnt = 0
    for sentence in sentences:
        for word in jieba.analyse.extract_tags(sentence, topK=TopkeyCnt):
            for i in range(typecnt):
                if word in keywordlist[i]:
                    X[cnt][i] += 1
        cnt += 1
    return X

In [ ]:
featuresB = TextToKeyVector(text_data)
labelsB = torch.tensor(score_data,dtype=torch.float)
data_iterB = load_array((featuresB, labelsB), batch_size)

In [ ]:
netB = nn.Sequential(nn.Linear(typecnt, 1))
lossB = nn.MSELoss()
trainerB = torch.optim.SGD(netB.parameters(), lr=0.03)

In [ ]:
sum_epochs_key = 3
for epoch in range(sum_epochs_key):
    for X, y in data_iterB:
        l = lossB(netB(X) ,y)
        trainerB.zero_grad()
        l.backward()
        trainerB.step()
    l = lossB(netB(featuresB), labelsB)
    print(f'epoch {epoch + 1}, loss {l:f}')

In [ ]:
print(LrTest(netB,lossB,featuresB,labelsB))

In [ ]:
weight_key = 0.6
def GetScore(featuresA,featuresB):
    return netA(featuresA) * (1-weight_key) + netB(featuresB) * weight_key